In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import seaborn as sns

from sklearn.model_selection import KFold, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMRegressor
from tqdm import tqdm
import optuna
from functools import partial
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [54]:
t1=[[1,1,1],[3,3,3]]
t2=[[1,1,1],[3,3,3]]
test_t=[]
test_t[len(test_t):] = t1
test_t[len(test_t):] = t2
np.save(f'test.npy', test_t)
test_l = np.load('test.npy')
np.array(test_l).mean(axis=0)

array([2., 2., 2.])

In [55]:
train = pd.read_csv('..\\kaggle_data\\train.csv')
test = pd.read_csv('..\\kaggle_data\\test.csv')

In [56]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train.target
X_test = test.drop(['id'], axis=1)

In [57]:
cat_cols = [feature for feature in train.columns if 'cat' in feature]

def label_encoder(df):
    for feature in cat_cols:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

In [58]:
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

In [59]:
split = KFold(n_splits=5)

In [60]:
preds_list_final_iterations =[]

In [63]:
root_directory = 'N:\\kaggle_data\\march'

def objective(trial, X, y, name='xgb'):
            
    if trial.number % 5 == 0:
        clear_output(wait=True)
    
    params = {'max_depth':trial.suggest_int('max_depth', 5, 50),
              'n_estimators':200000,
              #'boosting':trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
              'subsample': trial.suggest_uniform('subsample', 0.2, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.2, 1.0),
              'learning_rate':trial.suggest_uniform('learning_rate', 0.007, 0.02),
              'reg_lambda':trial.suggest_uniform('reg_lambda', 0.01, 50),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 0.01, 50),
              'min_child_samples':trial.suggest_int('min_child_samples', 5, 100),
              'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 300, 1000),
              'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)}

    model = LGBMRegressor(**params)
                  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
    
    print(f'Fit Starting')
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
              eval_metric=['rmse'],
              early_stopping_rounds=250, 
              categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
              verbose=0)
    
    print(f'Appending to List')
    preds_list_final_iterations.append(model.predict(X_test))

    dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    #np.save(f'..\\kaggle_data\\{dt_string}_FI{fold_index}_sub.npy', preds_list_final_iterations)
    print(f'Calculating Mean')
    y_preds_final_iteration = np.array(preds_list_final_iterations).mean(axis=0)
    print(f'length of infal list:{len(preds_list_final_iterations)}')
    submission = pd.DataFrame({'id':test.id,
              'target':y_preds_final_iteration})
    file_target = f'{root_directory}\\wave5_{dt_string}_AUTOML_sub-40.csv'
    submission.to_csv(file_target, index=False)
    print(f'written to CSV {file_target}')
    
    #train_score = np.round(np.sqrt(mean_squared_error(y_train, model.predict(X_train))), 15)
    test_score = np.round(np.sqrt(mean_squared_error(y_val, model.predict(X_val))), 15)
                  
    print(f'TEST RMSE : {test_score}')
                  
    return test_score

In [64]:
optimize = partial(objective, X=X_train, y=y_train)

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(optimize, n_trials=300)

best_trial = study_lgbm.best_trial.params

clear_output(wait=True)

print(f'best trial: {best_trial}')
# i have commented out the trials so as to cut short the notebook execution time.

best trial: {'max_depth': 33, 'subsample': 0.2028128133863628, 'colsample_bytree': 0.3316256936017043, 'learning_rate': 0.007295415117417998, 'reg_lambda': 21.211226081349032, 'reg_alpha': 7.386690008287902, 'min_child_samples': 88, 'num_leaves': 200, 'max_bin': 859, 'cat_smooth': 100, 'cat_l2': 3.5709736612307674}


In [ ]:
y_preds_final_iteration = np.array(preds_list_final_iterations[-1:]).mean(axis=0)
submission = pd.DataFrame({'id':test.id,
          'target':y_preds_final_iteration})
submission.to_csv(f'..\\kaggle_data\\{dt_string}_AUTOML_sub.csv', index=False)

In [ ]:
fold_ittr_range = 20
preds_list_final_iterations =[]

for fold_index in range(2, fold_ittr_range):
    print(f'Current fold index {fold_index}')
    split = KFold(n_splits=fold_index)
    lgbm_params = {'max_depth': 33, 'subsample': 0.2028128133863628, 'colsample_bytree': 0.3316256936017043, 'learning_rate': 0.007295415117417998, 'reg_lambda': 21.211226081349032, 'reg_alpha': 7.386690008287902, 'min_child_samples': 88, 'num_leaves': 200, 'max_bin': 859, 'cat_smooth': 100, 'cat_l2': 3.5709736612307674,
                'verbose':-1}
    

    preds_list_base = []
    preds_list_final_iteration = []
    preds_list_all = []

    for train_idx, val_idx in tqdm(split.split(X_train)):
        X_tr = X_train.iloc[train_idx]
        X_val = X_train.iloc[val_idx]
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]

        Model = LGBMRegressor(**lgbm_params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                      eval_metric=['rmse'],
                      early_stopping_rounds=250, 
                      categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                      #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
                      verbose=0)

        preds_list_base.append(Model.predict(X_test))
        preds_list_all.append(Model.predict(X_test))
        print(f'RMSE for Base model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
        first_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
        params = lgbm_params.copy()

        for i in range(1, 8):
            if i >2:    

                # reducing regularizing params if 

                params['reg_lambda'] *= 0.9
                params['reg_alpha'] *= 0.9
                params['num_leaves'] += 40

            params['learning_rate'] = 0.003
            Model = LGBMRegressor(**params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                      eval_metric=['rmse'],
                      early_stopping_rounds=200, 
                      categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                      #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
                      verbose=0,
                      init_model=Model)

            preds_list_all.append(Model.predict(X_test))
            print(f'RMSE for Incremental trial {i} model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
        last_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
        print('',end='\n\n')
        print(f'Improvement of : {first_rmse - last_rmse}')
        print('-' * 100)
        preds_list_final_iteration.append(Model.predict(X_test))
    preds_list_final_iterations[len(preds_list_final_iterations):] = preds_list_final_iteration
    dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    np.save(f'..\\kaggle_data\\{dt_string}_FI{fold_index}_sub.npy', preds_list_final_iterations)
    y_preds_final_iteration = np.array(preds_list_final_iterations).mean(axis=0)
    submission = pd.DataFrame({'id':test.id,
              'target':y_preds_final_iteration})
    submission.to_csv(f'..\\kaggle_data\\{dt_string}_FI{fold_index}_sub.csv', index=False)